# Outlier detection with LOF(Local Outlier Factor)

Based on the clickstream event frequency pattern in Q2Q3_input.csv, apply LOF algorithm to
calculate LOF for each point with the following initial settings:
1. Set k = 2 and use Manhattan distance. 
2. Set k = 3 and use Euclidean distance.
3. Report top 5 outliers 


In [ ]:
#python 2.7
%matplotlib inline
%pylab inline
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.spatial.distance import pdist, squareform

Populating the interactive namespace from numpy and matplotlib


In [ ]:
data_input = pd.read_csv('https://raw.githubusercontent.com/gulabpatel/Statistics/master/Dataset/Q2Q3_input.csv')

In [ ]:
data_input.head()

,user_id,load_video,pause_video,play_video,seek_video,speed_change_video,stop_video
0,0,2.0,1.0,4.0,1.0,0.0,1.0
1,1,6.0,14.0,14.0,0.0,0.0,1.0
2,2,1.0,0.0,0.0,0.0,0.0,0.0
3,3,2.0,2.0,2.0,0.0,0.0,1.0
4,4,1.0,3.0,22.0,18.0,0.0,0.0


In [ ]:
data_input.iloc[1][1]

6.0

In [ ]:
#Reachdist function
def reachdist(distance_df, observation, index):
    return distance_df[observation][index]

In [ ]:
#LOF algorithm implementation from scratch
def LOF_algorithm(data_input, distance_metric = "cityblock", p = 5):
    distances = pdist(data_input.values, metric=distance_metric)
    dist_matrix = squareform(distances)
    distance_df = pd.DataFrame(dist_matrix)
    
    k = 2 if distance_metric == "cityblock" else 3 
    observations = distance_df.columns
    lrd_dict = {}
    n_dist_index = {}
    reach_array_dict = {}
    
    for observation in observations:
        dist = distance_df[observation].nsmallest(k+1).iloc[k]
        indexes = distance_df[distance_df[observation] <= dist].drop(observation).index
        n_dist_index[observation] = indexes
    
        reach_dist_array = []
        for index in indexes:
            #make a function reachdist(observation, index)
            dist_between_observation_and_index = reachdist(distance_df, observation, index)
            dist_index =  distance_df[index].nsmallest(k+1).iloc[k]
            reach_dist = max(dist_index, dist_between_observation_and_index)
            reach_dist_array.append(reach_dist)
        lrd_observation = len(indexes)/sum(reach_dist_array)
        reach_array_dict[observation] = reach_dist_array
        lrd_dict[observation] = lrd_observation
        
    #Calculate LOF
    LOF_dict = {}
    for observation in observations:
        lrd_array = []
        for index in n_dist_index[observation]:
            lrd_array.append(lrd_dict[index])
        LOF = sum(lrd_array)*sum(reach_array_dict[observation])/np.square(len(n_dist_index[observation]))
        LOF_dict[observation] = LOF

    return sorted(LOF_dict.items(), key=lambda x: x[1], reverse=True)[:p]

In [ ]:
LOF_algorithm(data_input, p = 5)

[(19, 11.07),
 (525, 8.867228661749209),
 (66, 5.026785714285714),
 (638, 4.334727219682972),
 (177, 3.6292633292633294)]

In [ ]:
LOF_algorithm(data_input, p = 5, distance_metric = 'euclidean')

[(638, 3.0800716645705695),
 (525, 3.010316256261629),
 (19, 2.8402916620868903),
 (66, 2.801410266169121),
 (65, 2.6456528412196416)]

----------------------